# Holding Analysis Tool - data extraction for 100+ BPIDs - 2020-01-30

Final report: https://docs.google.com/document/d/1-HARyTVdBEioB72c22oVXzr_-tbw12EDQvaEFiQZVKE/edit#heading=h.b8ff86bcbbxs

## Customers IDs 

* BPIDs for prototype from Markus https://docs.google.com/spreadsheets/d/1bIRQxULsSkGrtVRd63-GcgM67NSFsScPa6VC0z5xYf0/edit#gid=0
* Site to BP ID mapping table https://hive.springernature.com/docs/DOC-185186

## APIs: BPIDs hierarchy

* api_parents = "http://institution-service.live.cf.private.springer.com/v1/hierarchy/{}"
* api_children = "http://institution-service.live.cf.private.springer.com/v1/children/{}"

## APIs: KBARTS

* url_books = "http://ml-read-online.live.sl.i.springer.com:7655/admin-portal/201802052054-134/kbart/book-report.xqy?bp={}"
* url_journals = "http://ml-read-online.live.sl.i.springer.com:7655/admin-portal/201802052054-134/kbart/journal-report.xqy?bp={}"
* url_nature = "http://entitlements-api.live.private.springernature.io/entitlements/v1/journals/kbart?bpid={}"

## APIs: Usage on GBQ

* online console https://console.cloud.google.com/bigquery?project=usage-data-reporting&organizationId=271269911051
* hive docs https://hive.springernature.com/groups/sap-business-intelligence/pages/google-bigquery-support
* query for Springer usage (using BPIDs) is:

```
Select DOI, Usage_2016, Usage_2017, Usage_2018, Denials_2016, Denials_2017, Denials_2018 FROM (
    SELECT 
    BP_ID, Item_DOI as DOI,  sum(clicks),
    sum(Case when Calendar_Year = 2016 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2016,
    sum(Case when Calendar_Year = 2017 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2017,
    sum(Case when Calendar_Year = 2018 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2018,
    sum(Case when Calendar_Year = 2016 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2016,
    sum(Case when Calendar_Year = 2017 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2017,
    sum(Case when Calendar_Year = 2018 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2018,

    FROM `usage-data-reporting.PROD_IFM_Usage.mv_usage_bp_monthly_spl` 
    Where 
    Document_Type = 'ARTICLE'
    and Item_DOI is not null
    and DOI is not null
    and clicks is not null
    and Calendar_Year in (2016, 2017, 2018)
    Group by 1, 2
    Order by 1, 2)
Where BP_ID = '{}' 
```
* query for Nature usage, using Site IDs, is:

```
Select DOI, Usage_2016, Usage_2017, Usage_2018, Denials_2016, Denials_2017, Denials_2018 FROM (
 
SELECT
Site_ID, Item_DOI as DOI,  sum(clicks),
 
sum(Case when Calendar_Year = 2016 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2016,
sum(Case when Calendar_Year = 2017 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2017,
sum(Case when Calendar_Year = 2018 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2018,
sum(Case when Calendar_Year = 2016 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2016,
sum(Case when Calendar_Year = 2017 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2017,
sum(Case when Calendar_Year = 2018 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2018,
 
FROM `usage-data-reporting.PROD_IFM_Usage.mv_usage_bp_monthly_nat`
Where
Item_DOI is not null
and DOI is not null
and clicks is not null
and Calendar_Year in (2016, 2017, 2018)
Group by 1, 2
Order by 1, 2)
 
Where Site_ID = '2000323035'
```

## 1. Load Customer IDs 

In [2]:
import requests
import pandas as pd
import json
import os
from tqdm.notebook import tqdm as progress
from google.cloud import bigquery

In [3]:
bpids = pd.read_csv("data_input/Markus - HA BPIDs for MVP - Sheet1.csv")
BPIDS_LIST = bpids["BPID"].to_list()
bpids.head()

,BPID,Account,Contact
0,3000612920,Research Organization of Information,Martijn Timman
1,2000703964,INSERM DISC-IST,Martijn Timman
2,2000478521,The University of Tokyo,Martijn Timman
3,2000277680,Shanghai Library / Institute of,Martijn Timman
4,2000342845,National Library of China NLC,Martijn Timman


In [4]:
siteids = pd.read_csv("data_input/Site to BP ID mapping table_7 Jan 20.csv")

def bpid_to_siteid(bpid):
    try:
        return siteids[siteids['Business Partner'] == bpid]['Identification number (= Site ID)'].values[0]
    except:
        return None
    
if False:
    for x in sorted(BPIDS_LIST):
        print(bpid_to_siteid(x))
        
# len([x for x in BPIDS_LIST if bpid_to_siteid(x)])
# => 154 with mappings to site IDs
siteids.head()

,Business Partner,Identification Type,Identification number (= Site ID),BP Type
0,1000057105,ZSITE,5054,4
1,1000194767,ZSITE,7564,4
2,1000207511,ZSITE,17293,4
3,1000213645,ZSITE,251,4
4,1000213969,ZSITE,14052,4


In [5]:
OUTPUT_FOLDER = "/Users/michele.pasin/Desktop/data_output"

OUTPUT_FOLDER = "/Volumes/Albano/2020-01-31-holding-analysis-extraction"

def gen_bpid_folder(folder, top_level_folder=OUTPUT_FOLDER, docreate=True):
    if not os.path.exists(top_level_folder):
        if docreate: os.makedirs(top_level_folder)
    if not os.path.exists(top_level_folder + "/" + folder):
        if docreate: os.makedirs(top_level_folder + "/" + folder)
    return top_level_folder + "/" + folder + "/"



## 2. Extracting all parents and children

* from a single BPID, pull out all parents and children as JSON
* save JSON to folder, if needed
* return simple list of BPIDs, including original one

PS
* `extract_related_bpids_from_local_file` is a modified version that uses previously saved local data

In [10]:
api_parents = "http://institution-service.live.cf.private.springer.com/v1/hierarchy/{}"
api_children = "http://institution-service.live.cf.private.springer.com/v1/children/{}"

        
def extract_related_bpids(BPID):
    BPID = str(BPID)
    print("======\nChecking Hierarchy for BPID: ", BPID, "\n======")
    path = gen_bpid_folder(BPID)
    # call API
    parents = requests.get(api_parents.format(BPID)).json()
    children = requests.get(api_children.format(BPID)).json()
    # save
    with open(path + 'parents.json', 'w') as outfile:
        json.dump(parents, outfile)
    with open(path + 'children.json', 'w') as outfile:
        json.dump(children, outfile)
    print("..data saved in", path)
    # extract BPIDs as list
    bpid_list = [BPID] # seed with main bpid
    for x in parents["parents"]: 
        bpid_list.append(x['institution']['id'])
        if x['parents']:  # just go one level up
            for z in x["parents"]: 
                bpid_list.append(z['institution']['id'])            
    for x in children:
        bpid_list.append(x['id'])
    print("Related BPIDs found: ", len(bpid_list), "=", str(bpid_list))
    return bpid_list


def extract_related_bpids_from_local_file(BPID):
    """use the previously saved json data (from extract_related_bpids) instead of calling SN API"""
    BPID = str(BPID)
    print("======\nChecking Pre-saved Hierarchy for BPID: ", BPID, "\n======")
    path = gen_bpid_folder(BPID, docreate=False)
    # get json data
    with open(path + 'parents.json', "r") as read_file:
        parents = json.load(read_file)
    with open(path + 'children.json', "r") as read_file:
        children = json.load(read_file)
    bpid_list = []
    # extract BPIDs as list
    bpid_list = [BPID] # seed with main bpid
    for x in parents["parents"]: 
        bpid_list.append(x['institution']['id'])
        if x['parents']:  # just go one level up
            for z in x["parents"]: 
                bpid_list.append(z['institution']['id'])            
    for x in children:
        bpid_list.append(x['id'])
    print("Related BPIDs found: ", len(bpid_list), "=", str(bpid_list))
    return bpid_list



# eg
if False:
    for x in progress(BPIDS_LIST[:10]):
        extract_related_bpids(x)
        
# eg
if False:
    for x in progress(BPIDS_LIST[:10]):
        extract_related_bpids_from_local_file(x)

## 3. Extracting KBARTs

* for a single BPID, get nature/springer kbarts (books is postponed)
* save all in the main BPID folder (= customer the analysis focuses on)

In [76]:
url_books = "http://ml-read-online.live.sl.i.springer.com:7655/admin-portal/201802052054-134/kbart/book-report.xqy?bp={}"
url_journals = "http://ml-read-online.live.sl.i.springer.com:7655/admin-portal/201802052054-134/kbart/journal-report.xqy?bp={}"
url_nature = "http://entitlements-api.live.private.springernature.io/entitlements/v1/journals/kbart?bpid={}"


def extract_kbarts(BPID, BPID_TOP=None):
    if not BPID_TOP:
        BPID_TOP = BPID
    BPID, BPID_TOP = str(BPID), str(BPID_TOP)
    path = gen_bpid_folder(BPID_TOP)
    print("===\nExtracting KBARTs for", BPID, " -- top level=", BPID_TOP)
    if False:
        # not needed yet
        print("..books...")
        d = requests.get(url_books.format(BPID), auth=('casper-db-user', 'casper'))
        with open(path + BPID + "_books.kbart", 'wb') as outfile:
            outfile.write(d.text.encode('utf8'))
    #
    print("..journals/springer...")
    d = requests.get(url_journals.format(BPID), auth=('casper-db-user', 'casper'))
    with open(path + BPID + "_journals_springer.kbart", 'wb') as outfile:
        outfile.write(d.text.encode('utf8'))
    #
    print("..journals/nature...")
    d = requests.get(url_nature.format(BPID), auth=('casper-db-user', 'casper'))
    with open(path + BPID + "_journals_nature.kbart", 'wb') as outfile:
        outfile.write(d.text.encode('utf8'))

        
# eg
if False:
    for x in progress(BPIDS_LIST[:10]):
        extract_kbarts(x)


## 4. Running the Extraction: kbarts and bpids parents/children


In [ ]:
source_data = BPIDS_LIST

errors = []
for b1 in progress(source_data):
    try:
        related = extract_related_bpids(b1)
    except:
        related = []
        errors.append(b1)
        print("++++ ERROR ++++")
        print("Errors index =", errors)
    for b2 in progress(related):
        extract_kbarts(b2, b1)

print("**** completed ****")
print("Errors index =", errors)

## 5. Extracting Usage: Springer

* requires a GBQ service account credentials (JSON file)
* for a BPID, pull all usage data - Springer only for the moment
* if a BPID is not found or not valid, it simply outputs a zero-lines CSV


In [9]:
from google.cloud import bigquery

credentials_file = '/Users/michele.pasin/Google Drive/Colab Notebooks/Reports/SpringerNature/2020-01-SN-Markus-BPIDs/data_input/usage-data-reporting-523fb677bd0a.json'
project_id = 'usage-data-reporting'
# Explicitly use service account credentials by specifying the private key file
client_ref = bigquery.Client.from_service_account_json(credentials_file)

gbq_query = '''
Select DOI, Usage_2016, Usage_2017, Usage_2018, Denials_2016, Denials_2017, Denials_2018 FROM (
    SELECT 
    BP_ID, Item_DOI as DOI,  sum(clicks),
    sum(Case when Calendar_Year = 2016 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2016,
    sum(Case when Calendar_Year = 2017 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2017,
    sum(Case when Calendar_Year = 2018 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2018,
    sum(Case when Calendar_Year = 2016 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2016,
    sum(Case when Calendar_Year = 2017 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2017,
    sum(Case when Calendar_Year = 2018 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2018,

    FROM `usage-data-reporting.PROD_IFM_Usage.mv_usage_bp_monthly_spl` 
    Where 
    Document_Type = 'ARTICLE'
    and Item_DOI is not null
    and DOI is not null
    and clicks is not null
    and Calendar_Year in (2016, 2017, 2018)
    Group by 1, 2
    Order by 1, 2)
Where BP_ID = '{}' 
# limit 100
'''


def extract_usage_springer(BPID, BPID_TOP=None):
    if not BPID_TOP:
        BPID_TOP = BPID
    BPID, BPID_TOP = str(BPID), str(BPID_TOP)
    path = gen_bpid_folder(BPID_TOP)
    print("===\nExtracting Springer Usage for", BPID, " -- top level=", BPID_TOP)
    #
    query_job = client_ref.query(gbq_query.format(BPID)).to_dataframe()
    print("..saving..")
    query_job.to_csv(path + BPID + "_usage_springer.csv", index=False)


        
# eg
if False:
    for x in progress(BPIDS_LIST[:1]):
        extract_usage_springer(x)

        

## 6. Running the Extraction: Usage Springer

* first, an extra function to reuse the BPIDs file structure from before to know which BPIDs to use


In [ ]:
source_data = BPIDS_LIST
errors = []

for b1 in progress(source_data):
    try:
        related = extract_related_bpids_from_local_file(b1)
    except:
        related = []
        errors.append(b1)
        print("++++ ERROR ++++")
        print("Errors index =", errors)
    for b2 in progress(related):
        extract_usage_springer(b2, b1)

print("**** completed ****")
print("Errors index =", errors)

####  QAing the data: Springer Extraction

* folders created VS BPIDs in Markus list (192)

In [6]:
len(BPIDS_LIST)

192

In [12]:
path = "/Users/michele.pasin/Desktop/data_output"
folders = [f for f in os.listdir(path) if not f.startswith('.')]
len(folders)

189

In [21]:
len([x for x in BPIDS_LIST if str(x)  in folders])

192

In [22]:
len(set(BPIDS_LIST))

189

Turns out the original list contained 3 sets of duplicates.

## 7. Extracting Usage: Nature

* requires a GBQ service account credentials (JSON file)
* for each BPID, find the corresponding SITE_ID
* for a SITE_ID, pull all usage data - using Nature table


In [1]:
from google.cloud import bigquery

credentials_file = '/Users/michele.pasin/Google Drive/Colab Notebooks/Reports/SpringerNature/2020-01-SN-Markus-BPIDs/data_input/usage-data-reporting-523fb677bd0a.json'
project_id = 'usage-data-reporting'
# Explicitly use service account credentials by specifying the private key file
client_ref = bigquery.Client.from_service_account_json(credentials_file)

gbq_query = '''
Select DOI, Usage_2016, Usage_2017, Usage_2018, Denials_2016, Denials_2017, Denials_2018 FROM (

SELECT
Site_ID, Item_DOI as DOI,  sum(clicks),

sum(Case when Calendar_Year = 2016 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2016,
sum(Case when Calendar_Year = 2017 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2017,
sum(Case when Calendar_Year = 2018 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2018,
sum(Case when Calendar_Year = 2016 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2016,
sum(Case when Calendar_Year = 2017 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2017,
sum(Case when Calendar_Year = 2018 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2018,

FROM `usage-data-reporting.PROD_IFM_Usage.mv_usage_bp_monthly_nat`
Where
Item_DOI is not null
and DOI is not null
and clicks is not null
and Calendar_Year in (2016, 2017, 2018)
Group by 1, 2
Order by 1, 2)

Where Site_ID = '{}'
'''


def extract_usage_nature(BPID, BPID_TOP=None):
    if not BPID_TOP:
        BPID_TOP = BPID
    SITE_ID = bpid_to_siteid(int(BPID))
    BPID, BPID_TOP = str(BPID), str(BPID_TOP)
    path = gen_bpid_folder(BPID_TOP)
    if SITE_ID:
        print("===\nExtracting Nature Usage for", BPID, " -- top level=", BPID_TOP, "Site_id=", SITE_ID)
        #
        query_job = client_ref.query(gbq_query.format(str(SITE_ID))).to_dataframe()
        print("..saving..")
        query_job.to_csv(path + BPID + "_usage_nature.csv", index=False)
    else:
        print("...mapping from", BPID, " to Site_id not found: skipping")

        
# eg
if False:
    for x in progress(BPIDS_LIST[:10]):
        extract_usage_nature(x)

        

## 8. Running the Extraction: Usage Nature


In [11]:
source_data = BPIDS_LIST
errors = []

for b1 in progress(source_data):
    try:
        related = extract_related_bpids_from_local_file(b1)
    except:
        related = []
        errors.append(b1)
        print("++++ ERROR ++++")
        print("Errors index =", errors)
    # if BPIDs are found
    for b2 in progress(related):
        extract_usage_nature(b2, b1)

print("**** completed ****")
print("Errors index =", errors)

Checking Pre-saved Hierarchy for BPID:  3000612920 
Related BPIDs found:  2 = ['3000612920', '3003777848']


...mapping from 3000612920  to Site_id not found: skipping
===
Extracting Nature Usage for 3003777848  -- top level= 3000612920 Site_id= 13251
..saving..

Checking Pre-saved Hierarchy for BPID:  2000703964 
Related BPIDs found:  1 = ['2000703964']


...mapping from 2000703964  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  2000478521 
Related BPIDs found:  2 = ['2000478521', '3000675571']


===
Extracting Nature Usage for 2000478521  -- top level= 2000478521 Site_id= 4604
..saving..
...mapping from 3000675571  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  2000277680 
Related BPIDs found:  8 = ['2000277680', '3000160641', '3000507429', '3991438065', '3001358220', '3002205109', '3002600051', '3002642964']


===
Extracting Nature Usage for 2000277680  -- top level= 2000277680 Site_id= 14945
..saving..
...mapping from 3000160641  to Site_id not found: skipping
...mapping from 3000507429  to Site_id not found: skipping
...mapping from 3991438065  to Site_id not found: skipping
...mapping from 3001358220  to Site_id not found: skipping
...mapping from 3002205109  to Site_id not found: skipping
...mapping from 3002600051  to Site_id not found: skipping
...mapping from 3002642964  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  2000342845 
Related BPIDs found:  5 = ['2000342845', '3000202650', '3000508393', '3000595123', '3000701760']


...

===
Extracting Nature Usage for 3000142567  -- top level= 3000142567 Site_id= 1558
..saving..
...mapping from 8200828607  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  8200178197 
Related BPIDs found:  2 = ['8200178197', '8200828607']


===
Extracting Nature Usage for 8200178197  -- top level= 8200178197 Site_id= 9660
..saving..
...mapping from 8200828607  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  8200699471 
Related BPIDs found:  2 = ['8200699471', '8200828607']


===
Extracting Nature Usage for 8200699471  -- top level= 8200699471 Site_id= 4474
..saving..
...mapping from 8200828607  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  8200711893 
Related BPIDs found:  2 = ['8200711893', '8200828607']


===
Extracting Nature Usage for 8200711893  -- top level= 8200711893 Site_id= 1566
..saving..
...mapping from 8200828607  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  8200930480 
Related BPIDs found:  2 = ['8200930480', '8200828607']


===
Extracting Nature Usage for 8200930480  -- top level= 8200930480 Site_id= 1282
..saving..
...mapping from 8200828607  to Site_id not found: skipping


**** completed ****
Errors index = [30036927080, 3003695120, 3003343017, 3000633321, 2000341714, 8200769361]
